In [1]:
# implement MDS dimension reduction algorithm and then using scikit-learn algorithm KNN to fulfill anormaly detection
# algorithm

In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import manifold

In [3]:
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [4]:
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [5]:
# df = collection_read_mongo(collection="training")

df = pd.read_csv("training.csv")

In [6]:
df[:3]

,Time,UserID,LocationX,LocationY,RSRP1,RSRQ1,RSRP2,RSRQ2,RSRP3,RSRQ3,RSRP4,RSRQ4,Label
0,0.2,1,1165.135603,157.155659,-71.781608,NaN,-76.114469,NaN,-76.421107,NaN,-77.007771,NaN,0
1,0.2,2,18.101453,97.904367,-69.099606,NaN,-71.002579,NaN,-74.648307,NaN,-76.255106,NaN,0
2,0.2,3,539.327246,151.255141,-67.527468,NaN,-69.966291,NaN,-70.339135,NaN,-74.413793,NaN,0


In [8]:
df.dropna(axis=0, how="any", inplace=True)
df.drop(df.columns[:4], axis=1, inplace=True)
y = df["Label"]
X = df.drop(df.columns[-1], axis=1)
# apply MDS dimension reduction to signalDF, with only 3 dimensions left
mds = manifold.MDS(3, max_iter=200, n_init=1)
import time
startTime = time.time()
threeDimSig = mds.fit_transform(X)
print("elapse time: ", (time.time()-startTime))

elapse time:  721.722332239151


In [ ]:
data.shape

In [ ]:
data[:3]

In [ ]:
type(data)

In [ ]:
type(data["Time"][0])

In [ ]:
testData = data[:10000]

In [ ]:
# for each user A, we pick the top 4 highest RSRP, RSRQ value at a time point t.
identiferList = list()
pd.DataFrame(columns=["Time", "UserID"])
signalList = list()

for ident, group in testData.groupby(["Time", "UserID"]):
#     iterate the group object, and pick the top 4 highest rsrp value and then its corresponsing rsrq value in that row
#     print(type(group))
    top4Row = group.sort_values(by=["RSRP"])[:4]
#     print(top4Row)
    try:
        signalRow = [top4Row.iloc[0]["RSRP"], top4Row.iloc[0]["RSRQ"], top4Row.iloc[1]["RSRP"], top4Row.iloc[1]["RSRQ"],top4Row.iloc[2]["RSRP"], top4Row.iloc[2]["RSRQ"], top4Row.iloc[3]["RSRP"], top4Row.iloc[3]["RSRQ"]]
        signalList.append(signalRow)
        ident = (round(ident[0], 1), ident[1])
        identiferList.append(ident)
    except:
        pass
    

In [ ]:
signalDF = pd.DataFrame(signalList, columns=["RSRP_1","RSRQ_1","RSRP_2","RSRQ_2","RSRP_3","RSRQ_3","RSRP_4","RSRQ_4"])

In [ ]:
signalDF[:3]

In [ ]:
identiferDF = pd.DataFrame(identiferList, columns=["Time", "UserID"])

In [ ]:
identiferDF[:3]

In [ ]:
identiferDF.shape

In [ ]:
signalDF.shape

In [ ]:
# create a new Dataframe with merging two exist Dataframe, and length of these two dataframe is same
referenceDF = identiferDF.merge(signalDF, left_index=True, right_index=True)

In [ ]:
referenceDF[:5]

In [ ]:
# apply MDS dimension reduction to signalDF, with only 3 dimensions left
mds = manifold.MDS(3, max_iter=200, n_init=1)


In [ ]:
# before applying MDS, need to remove rows which contain NaN value, so that MDS can run correctly.
import time
signalDF = signalDF.dropna(axis=0, how="any")

In [ ]:
signalDF[:3]

In [ ]:

startTime = time.time()
threeDimSig = mds.fit_transform(signalDF)
print("elapse time: ", (time.time()-startTime))

In [ ]:
threeDimSigDF = pd.DataFrame(threeDimSig)
threeDimSigDF[:5]

In [ ]:
# create a new Dataframe with merging two exist Dataframe, and length of these two dataframe is same
referenceDF = identiferDF.merge(threeDimSigDF, left_index=True, right_index=True)

In [ ]:
# implement k-KNNAD algorithm, the reason we don't implement LOFAD algorithm is k-KNNAD can achieve higher accuracy,
#  auc score is about 0.91, while LOFAD is 0.85 according to the paper 
# "a cell outage management framework for dense heterogeneous networks"

referenceDF[:5]

In [ ]:
mds3D = MDS(n_components=3, max_iter=1000, eps=1e-7, random_state=seed, dissimilarity="euclidean", n_jobs=2)

In [ ]:
out = mds3D.fit_transform(cleanedWantedDF)

In [ ]:
out[:3]

In [ ]:
from mpl_toolkits import mplot3d
ax = plt.axes(projection="3d")

In [ ]:
ax.scatter3D(out[:,0], out[:,1], out[:,2])
ax.view_init(azim=70, elev=50)
plt.show()

In [ ]:

plt.scatter(out[:, 0], out[:, 1], out[:,2])
plt.show()

In [ ]:
# after getting reduced-dimensional data, then calculate the nearest neighbor iterately
# store the embedded measurement in csv file, reference database
np.savetxt('embedding.csv', out, delimiter=",")

In [ ]:
from IPython.display import Latex
Latex(r"""\begin{eqnarray}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\
\nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} & = 0 
\end{eqnarray}""")

In [ ]:
Latex(r"""\begin{eqnarray}
D_{k-NNAD}(x_{i}, k, D_{M}) = \frac{1}{N_{tr}} \sum_{N_{tr}}^{i=1} I(d_{t} <= d_{i}) \\
\end{eqnarry}""")

In [ ]:
# use cross validation method, k-fold, k = 10
from sklearn.model_selection import KFold
k = 10
kf = KFold(n_splits = k)
indicesList = list()
for train, test in kf.split(out):
    indicesList.append((train, test))


In [ ]:

N_tr = len(indicesList[0][0])+len(indicesList[0][1])
# k means the top k nearest neighbor
k = 50 
trainData = np.take(out, indicesList[0][0])
testData = np.take(out, indicesList[0][1])

# for each data in test dataset, calculate distance between it and each data point in train dataset
# pick the top k(k=50)
for i in testData:
    distList = list()
    for j in trainData:
        distance = np.sqrt(np.sum((i-j)**2, axis=1))
        distList.append(distance)
#     sort distList and then pick the top k(k =50) elems
    distList = sorted(distList, reverse=True)
    topKDist = distList[:k]
    
        
        


In [ ]:
pos.get_params()

In [ ]:
mds.get_params()

In [ ]:
def _connect_mongo():
    """ A util for making a connection to mongo """
    host = "localhost"
    port = 27017
    username = ""
    password = ""
    db = "5gopt"

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [ ]:
def collection_read_mongo(collection, query={}, no_id = True):
    db = _connect_mongo()
    cursor = db[collection].find(query)
    df = pd.DataFrame(list(cursor))

    if no_id:
        try:
            del df["_id"]
        except:
            pass
    return df

In [ ]:
data = collection_read_mongo(collection="main_file_with_UserTHR")

In [ ]:
data.shape

In [ ]:
data[:3]

In [ ]:
testData = data[:1000]

In [ ]:
groupedData = 

In [ ]:
max(data["Throughput"])

In [ ]:
groupedData = data.groupby(["Time", "CellID"])[["Throughput"]].sum()

In [ ]:
type(groupedData)

In [ ]:
groupedData[:5]

In [ ]:
df = groupedData.to_frame

In [ ]:
groupedData[36.2, 2]

In [ ]:
groupedData["Throughput"]